---

# CSCI 3202, Fall 2022
# Homework 4
# Due: Friday September 30, 2022 at 6:00 PM

<br> 

### Your name: Xinyi Lu

<br> 

---

Some useful packages and libraries:




In [1]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part II, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right length, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>
<center>$\frac{(1+number of heads)}{(2+number of flips)}$</center>

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [2]:
## Your code goes here.
## Your code needs to display the first 10 lines of your emission and transition tables, 
## and generate an output file with the complete tables, called Assignment8Output_Lastname_Firstname
## Your code also needs to be structured as an HMM class
class HMM:
    def __init__(self, data):
        df = pd.read_csv(data, header=None, sep = ' ')
        self.emission, self.transition, self.initial = self.BuildHMM(df)
        self.emission = self.LaplaceSmoothing(self.emission) # apply smoothing to emission prob table
        self.transition = self.LaplaceSmoothing(self.transition)
        emission = self.emission[['Xt', 'Et', 'prob']]
        transition = self.transition[['Xt', 'Xt+1', 'prob']]
        emission = emission.pivot(index='Xt', columns='Et', values='prob')
        transition = transition.pivot(index='Xt', columns='Xt+1', values='prob')
        
        print("Emission table: ")
        print(emission.head(10))
        print("Transition table: ")
        print(transition.head(10))
        
        #save to output
        writer = pd.ExcelWriter('HW4Output_Lu_Xinyi.xlsx', engine='xlsxwriter')
        emission.to_excel(writer, sheet_name='emission_probability')
        transition.to_excel(writer, sheet_name='transition_probability')
        self.initial.to_excel(writer, sheet_name='initial_probability')
        writer.save()
        
        #convert to nested dictionaries structure
        emission_d = {}
        transition_d = {}
        
        for index, row in self.emission.iterrows():
            Xt = row['Xt']
            Et = row['Et']
            prob = row['prob']
            
            if Xt not in emission_d:
                emission_d[Xt] = {Et: prob}
            else:
                emission_d[Xt][Et] = prob
    
        for index, row in self.transition.iterrows():
            current_s = row['Xt']
            next_s = row['Xt+1']
            prob = row['prob']
            
            if current_s not in transition_d:
                transition_d[current_s] = {next_s: prob}
                
            else:
                transition_d[current_s][next_s] = prob
        
        initial_d = {row['X0']: row['prob'] for index, row in self.initial.iterrows()}
        self.emission = emission_d
        self.transition = transition_d
        self.initial = initial_d
        
        
    def BuildHMM(self, data):
        df = data.copy()
        #rename columns
        df.columns = ['state', 'obs']
        states = df['state'].unique().tolist()
        states.sort()
        
        #find the emission probabilities
        df_state_count = df.groupby('state').size().reset_index().rename(columns={0:'count_state'})
        df_obs_state = df.groupby(['obs', 'state']).size().reset_index().rename(columns = {0:'count'})
        
        #get all combinations of states and obs
        combi = {'state': [], 'obs': []}
        for i in states:
            for j in states:
                combi['state'].append(i)
                combi['obs'].append(j)
        combi = pd.DataFrame(combi)
        
        df_emission = combi.merge(df_obs_state, how = 'left', on = ['state', 'obs'])
        df_emission = df_emission.merge(df_state_count, how = 'left', on = 'state')
        df_emission = df_emission.fillna(0)
        df_emission['emission_prob'] = df_emission['count']/df_emission['count_state']
        df_emission.rename(columns = {'state': 'Xt', 'obs': 'Et'}, inplace=True)
        
        #find the transition probabilities
        tran_states = df['state']
        states_pairs = {}
        for first, second in zip(tran_states, tran_states[1:]):
            pair = (first, second)
            if pair not in states_pairs:
                states_pairs[pair] = 1
            else:
                states_pairs[pair] += 1

        df_state = pd.DataFrame.from_dict(states_pairs, orient='index', columns=['count']).reset_index()
        df_state['Xt'] = df_state['index'].apply(lambda x: x[0])
        df_state['Xt+1'] = df_state['index'].apply(lambda x: x[1])
        df_state = combi.merge(df_state, how = 'left', left_on = ['state', 'obs'], right_on = ['Xt', 'Xt+1'])
        df_state = df_state.drop(columns = ['index', 'Xt', 'Xt+1'])
        df_state = df_state.rename(columns = {'state': 'Xt', 'obs': 'Xt+1'})
        df_state = df_state.fillna(0)
        df_state = df_state.merge(df_state_count, how = 'left', left_on = 'Xt', right_on = 'state')
        df_state['transition_prob'] = df_state['count']/df_state['count_state']
        
        
        #find the initial distributions
        df_initial = pd.DataFrame({'X0': states})
        df_initial['prob'] = 1/27
        
        return df_emission, df_state, df_initial
    
    def LaplaceSmoothing(self, df):
        df_new = df.copy()
        df_new['count'] = df_new['count'] + 1
        df_new['count_state'] = df_new['count_state'] + df_new['Xt'].nunique()
        df_new['prob'] = df_new['count']/df_new['count_state']
        return df_new 
    
    def get_emission(self):
        return self.emission
    
    def get_transition(self):
        return self.transition
    
    def get_initial(self):
        return self.initial

In [3]:
hmm = HMM('typos20.data')

Emission table: 
Et         _         a         b         c         d         e         f  \
Xt                                                                         
_   0.999044  0.000037  0.000037  0.000037  0.000037  0.000037  0.000037   
a   0.000101  0.794792  0.000101  0.000101  0.000101  0.000101  0.000101   
b   0.000507  0.000507  0.798886  0.000507  0.000507  0.000507  0.000507   
c   0.000220  0.000220  0.000220  0.793521  0.044954  0.000220  0.052887   
d   0.000233  0.000233  0.000233  0.041007  0.801025  0.040541  0.033784   
e   0.000059  0.000059  0.000059  0.000059  0.066348  0.800662  0.000059   
f   0.000319  0.000319  0.000319  0.033536  0.030981  0.000319  0.802619   
g   0.000388  0.000388  0.041925  0.000388  0.000388  0.000388  0.041537   
h   0.000159  0.000159  0.043699  0.000159  0.000159  0.000159  0.000159   
i   0.000098  0.000098  0.000098  0.000098  0.000098  0.000098  0.000098   

Et         g         h         i  ...         q         r         s   

In [4]:
hmm.get_emission()

{'_': {'_': 0.9990444689452407,
  'a': 3.6751194413818446e-05,
  'b': 3.6751194413818446e-05,
  'c': 3.6751194413818446e-05,
  'd': 3.6751194413818446e-05,
  'e': 3.6751194413818446e-05,
  'f': 3.6751194413818446e-05,
  'g': 3.6751194413818446e-05,
  'h': 3.6751194413818446e-05,
  'i': 3.6751194413818446e-05,
  'j': 3.6751194413818446e-05,
  'k': 3.6751194413818446e-05,
  'l': 3.6751194413818446e-05,
  'm': 3.6751194413818446e-05,
  'n': 3.6751194413818446e-05,
  'o': 3.6751194413818446e-05,
  'p': 3.6751194413818446e-05,
  'q': 3.6751194413818446e-05,
  'r': 3.6751194413818446e-05,
  's': 3.6751194413818446e-05,
  't': 3.6751194413818446e-05,
  'u': 3.6751194413818446e-05,
  'v': 3.6751194413818446e-05,
  'w': 3.6751194413818446e-05,
  'x': 3.6751194413818446e-05,
  'y': 3.6751194413818446e-05,
  'z': 3.6751194413818446e-05},
 'a': {'_': 0.0001009387301907742,
  'a': 0.794791561522156,
  'b': 0.0001009387301907742,
  'c': 0.0001009387301907742,
  'd': 0.0001009387301907742,
  'e': 0.0

In [5]:
hmm.get_transition()

{'_': {'_': 3.6751194413818446e-05,
  'a': 0.1051819184123484,
  'b': 0.04950385887541345,
  'c': 0.04152884968761485,
  'd': 0.028996692392502757,
  'e': 0.030871003307607496,
  'f': 0.03594266813671444,
  'g': 0.01686879823594267,
  'h': 0.035464902609334804,
  'i': 0.08063212054391768,
  'j': 0.002021315692760015,
  'k': 0.002793090775450202,
  'l': 0.02223447262036016,
  'm': 0.04035281146637266,
  'n': 0.026387357589121647,
  'o': 0.07824329290701948,
  'p': 0.05365674384417494,
  'q': 0.0012127894156560089,
  'r': 0.026203601617052555,
  's': 0.075303197353914,
  't': 0.17478868063212055,
  'u': 0.010216832047041529,
  'v': 0.0048144064682102165,
  'w': 0.05380374862183021,
  'x': 0.00014700477765527378,
  'y': 0.0027563395810363835,
  'z': 3.6751194413818446e-05},
 'a': {'_': 0.06762894922781872,
  'a': 0.0001009387301907742,
  'b': 0.022509336832542647,
  'c': 0.03906328858382962,
  'd': 0.02836378318360755,
  'e': 0.0002018774603815484,
  'f': 0.004239426668012516,
  'g': 0.02

In [6]:
hmm.get_initial()

{'_': 0.037037037037037035,
 'a': 0.037037037037037035,
 'b': 0.037037037037037035,
 'c': 0.037037037037037035,
 'd': 0.037037037037037035,
 'e': 0.037037037037037035,
 'f': 0.037037037037037035,
 'g': 0.037037037037037035,
 'h': 0.037037037037037035,
 'i': 0.037037037037037035,
 'j': 0.037037037037037035,
 'k': 0.037037037037037035,
 'l': 0.037037037037037035,
 'm': 0.037037037037037035,
 'n': 0.037037037037037035,
 'o': 0.037037037037037035,
 'p': 0.037037037037037035,
 'q': 0.037037037037037035,
 'r': 0.037037037037037035,
 's': 0.037037037037037035,
 't': 0.037037037037037035,
 'u': 0.037037037037037035,
 'v': 0.037037037037037035,
 'w': 0.037037037037037035,
 'x': 0.037037037037037035,
 'y': 0.037037037037037035,
 'z': 0.037037037037037035}


## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Canvas called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [7]:
## Your code goes here. You need to implement the viterbi algorithm, and display the output that your code generates
## You also need to display the error rate for your algorithm.
df = pd.read_csv('typos20Test.data', sep = ' ').reset_index()
df.rename(columns = {'index': 'state', '..': 'output'}, inplace=True)
df.head()

,state,output
0,i,i
1,n,n
2,t,t
3,r,r
4,o,o


In [8]:
test = df['output']
test

0        i
1        n
2        t
3        r
4        o
        ..
20059    o
20060    e
20061    n
20062    c
20063    e
Name: output, Length: 20064, dtype: object

In [9]:
def Viterbi(emission_d, transition_d, initial_d, test):
    nSamples = len(test)
    nStates = len(transition_d) # no of states
    T1 = [{}] * nSamples #each element stores the probability of the most likely path so far
    T2 = [''] * nSamples #each element stores the Xi of the most likely path so far
    
    #initialise the tracking tables from first observation
    first_obs = test[0]
    Pi = initial_d
    first_d = {}
    for i in initial_d:
        state = i
        first_d[state] = np.log(emission_d[i][first_obs]) + np.log(initial_d[i])
    T1[0] = first_d
    T2[0] = max(T1[0], key = T1[0].get)
    
    for i in range(1, nSamples):
        Et_1 = test[i]
        V = T1[i-1]
        Xt_1 = ''
        prob = float('-inf')
        for j in V:
            Xt = j
            curr_transition = transition_d[Xt]
            curr_emission = {key: emission_d[key][Et_1] for key in emission_d}
            new_d = {Xt_1: np.log(curr_emission[Xt_1]) + np.log(curr_transition[Xt_1]) + V[Xt_1] for Xt_1 in curr_transition}
            new_prob = max(new_d.values())
            new_letter = max(new_d, key = new_d.get)
            if new_prob > prob:
                prob = new_prob
                Xi_1 = new_letter
            T1[i][j] = new_prob
        T2[i] = Xi_1 
    return T2       

def ErrorRate(state, reconstructed):
    correct = 0
    wrong = 0
    for i in range(len(state)):
        if state[i] == reconstructed[i]:
            correct += 1
        else:
            wrong += 1
    return wrong*100/(wrong+correct)

In [10]:
reconstructed = Viterbi(hmm.get_emission(), hmm.get_transition(), hmm.get_initial(), test)
states = df['state'].tolist()
print(ErrorRate(states, reconstructed))

37.350478468899524


In [11]:
reconstructed_txt = ''.join(x for x in reconstructed)
states_txt = ''.join(x for x in states)

In [12]:
reconstructed_txt

'introducfilnithebineusrfialjrebolutninianebits_conaeauebces_bafebneeb_a_diaasrerproebthebyumab_racebtheuqhavebgrearohcincreased_theblutebesoectaneuqod_thiarpof_ia_ebi_libebiniaebanced_coibceies_bit_thehqhavebfeetanihuaeebsociett_havebmaeeblifebujfuoeilllng_havebeibiedred_humahcbeihgs_tojihcihgifida_havebled_tojaidespeeaeblaysnlloficalbsncfeding_knitnebthied_eoeod_tojlhhcicalbsuffecimg_as_eeolbanebhacebingoidfec_srberebdamagebonithebnaturalbeoeod_thebconcihied_debelolmebg_of_tebnilloguqeillbeofseb_thebsifuarioniif_eillbcerfainluqsuniecf_ytnanibeihgebtlbgreaterpibcignifies_snebincpict_greagerpdamagebonifhebnaturalbaleod_ot_eillbleobabluqleaebtijgrezterpsofiap_eiarupgiln_anebparcuollfucalbeifferonh_anc_ot_mayqleaebfljuncrewxed_pgusicalbsucfeeihhcebeb_oniacgaiced_coihgeies_thebinebareialbtedbnilogucaijsysrem_mayqsurviceboebut_nayqbeeai_doeb_uf_if_survices_if_nayqebebhiakluqacuiebeba_loa_lebelbof_phhcicalbanebpaycuoligucalbsufferibg_bif_onluqafferppassihg_theluguqa_lonh_amebferhqpaihculble

In [13]:
states_txt

'introduction_the_industrial_revolution_and_its_consequences_have_been_a_disaster_for_the_human_race_they_have_greatly_increased_the_life_expectancy_of_those_of_us_who_live_in_advanced_countries_but_they_have_destabilized_society_have_made_life_unfulfilling_have_subjected_human_beings_to_indignities_have_led_to_widespread_psychological_suffering_in_the_third_world_to_physical_suffering_as_well_and_have_inflicted_severe_damage_on_the_natural_world_the_continued_development_of_technology_will_worsen_the_situation_it_will_certainly_subject_human_beings_to_greater_indignities_and_inflict_greater_damage_on_the_natural_world_it_will_probably_lead_to_greater_social_disruption_and_psychological_suffering_and_it_may_lead_to_increased_physical_suffering_even_in_advanced_countries_the_industrial_technological_system_may_survive_or_it_may_break_down_if_it_survives_it_may_eventually_achieve_a_low_level_of_physical_and_psychological_suffering_but_only_after_passing_through_a_long_and_very_painful_pe

## Part III - Summarize your results
As part of this assignment, include a short report in your notebook describing your work. The report should contain the following:
### Purpose: 
Purpose should be clear. It doesn't need to be long, but it should be clear.
### Procedure:
##### Transition Probabilities
In this section explain the way you have generated the Transition probabilities and the data structure you used. <br>

##### Emission Probabilities
In this section explain the way you have generated the Emission probabilities and the data structure you used.

##### Viterbi Algorithm
In this section explain about the algorithm and the data structure you used for storing the values in the matrix with its back pointer.
### Data
Explain about the data you used, including the data included in the data set and any pre-processing you did to the data.
### Results
Explain your observations, including the sentences that you tested and their corresponding outputs. Were there any sentences where your algorithm did not work?


### Purpose: 
The purpose is to reconstruct the text from the observed text using the Viterbi algorithm. To achieve this, a hidden markov model with the laplaced smoothed transition, emission and initial probabilites is created. 

### Procedure:

**Transition Probabilities**

I form the pairs of (Xt, Xt+1) by looping through a tuple of list of Xt and Xt+1. Next, I count the number of occurrences of each pair by using a dictionary. Then, I convert this to a dataframe and merge in all combinations of Xt and Xt+1 and the count of Xt. To calculate the laplace smoothed probabilities, I added 1 to the count of occurrences of each pair, added 27 to the count of Xt and performed count of (Xt, Xt+1)/count of Xt. 

<br>
Finally, I converted the data structure to a nested dictionary with the key being Xt and the value is a dictionary of the Xt+1 and respective conditional probabilities. 

<br>

Eg. {'_': {'_': 3.6751194413818446e-05,
  'a': 0.1051819184123484,
  'b': 0.04950385887541345,
  'c': 0.04152884968761485,
  'd': 0.028996692392502757,
  'e': 0.030871003307607496,
  'f': 0.03594266813671444,
  'g': 0.01686879823594267,
  'h': 0.035464902609334804,
  'i': 0.08063212054391768, ...} <br>

P('-'|'-') =  3.6751194413818446e-05; P('a'|'-') = 0.1051819184123484

**Emission Probabilities**

First, I count the occurrence of pairs of (Xt, Et) using the groupby function. Then I merge in the dataframe containing all combinations of (Xt, Et) with the count of Xt. To calculate the laplace smoothed probability, I added 1 to the count of each pair of Xt, Et, added 27 to the count of Xt and then got the count of (Xt, Et)/count of Xt. 
<br>

I then converted the dataframe to a nested dictionary similar to the transition probabilities.
<br>

Eg. {'_': {'_': 0.9990444689452407,
  'a': 3.6751194413818446e-05,
  'b': 3.6751194413818446e-05,
  'c': 3.6751194413818446e-05,
  'd': 3.6751194413818446e-05,
  'e': 3.6751194413818446e-05,
  'f': 3.6751194413818446e-05,
  'g': 3.6751194413818446e-05,
  'h': 3.6751194413818446e-05,
  'i': 3.6751194413818446e-05,
  'j': 3.6751194413818446e-05, ...} <br>

P('-'|'-') =  0.9990; P('a'|'-') =3.6751194413818446e-05

**Viterbi Algorithm**
Given a sequence of observations, the Viterbi algorithm finds the most likely state sequence that generated those observations. <br>

I used 2 variables, T1 and T2 to store the values. 
T1 is a list of dictionaries. The keys of the dictionaries are the states of the text, while the values are the log probability of the state sequence when the Xi = key. 

T2 is a list where each element is the most likely state at time t. 

In my code, I first initialized T1[0] with the V matrix, which is a dictionary in my implementation. 
Then, I looped through all the observations and each state for each observation. Then I calculated the log probabilities for each possible sequence and found the Xt+1 state with the highest log probability. Each element in the nested dictionary in T1 is reassigned to their log probabilities. The most likely Xt+1 state is stored in T2. 
Finally, I return X2 to find the reconstructed text. I compare this list to the original states list to calculate the error rate. 


### Data

2 datasets were used. typos20.data was used to create the structure of the hidden markov model and typos20Test.data was used to test the Viterbi algorithm. As typos20.data has no headers, I renamed the first and second column names to state and obs respectively. While setting up the distribution tables, they were further renamed into Xt, Xt+1 and Et. To generate the output of the distribution tables, I first converted them to a dataframe with the index as Xt and column as Xt+1/Et. Then, an xlsx file containing 3 sheets for each table was generated. 

As I imported the typos20Test.data as a dataframe, I first renamed the columns to state and output and used the output column to test my Viterbi algorithm. 

### Results

By comparing the reconstructed text and original text, there is an error rate of 37%. Most of my _ characters in the original text are other characters in the reconstructed text, which contributed substantially to the high error rate. 